In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
games = pd.read_parquet('/content/drive/MyDrive/Dict/games_dict.parquet')

In [ ]:
df = pd.read_parquet('/content/drive/MyDrive/Dict/df_users_items_dict.parquet')

In [ ]:
# Abrir el archivo parquet en un dataframe llamado reviews
reviews = pd.read_parquet('/content/drive/MyDrive/Dict/user_reviews.parquet')

In [ ]:
# prompt: valores unicos de sentiment_analysis del dataframe reviews

unique_sentiment_values = reviews['sentiment_analysis'].unique()
print(unique_sentiment_values)


[2 1 0]


In [ ]:
# Contar la cantidad de cada valor en la columna 'review'
reviews['sentiment_analysis'].value_counts()

2    37657
1    11816
0     9832
Name: sentiment_analysis, dtype: int64

Usamos este metodo

In [ ]:
# Explotar la columna 'items' para que cada diccionario sea una fila independiente
df_explotado = df.explode('items').reset_index(drop=True)

# Normalizar los datos en la columna 'items' para expandir los diccionarios en columnas individuales
df_normalizado = pd.json_normalize(df_explotado['items'])

# Unir el DataFrame normalizado con el DataFrame original
df_final = pd.concat([df_explotado.drop(columns=['items']), df_normalizado], axis=1)

In [ ]:
df_final

,user_id,item_id,item_name,playtime_2weeks,playtime_forever
0,76561197970982479,10,Counter-Strike,0.0,6.0
1,76561197970982479,20,Team Fortress Classic,0.0,0.0
2,76561197970982479,30,Day of Defeat,0.0,7.0
3,76561197970982479,40,Deathmatch Classic,0.0,0.0
4,76561197970982479,50,Half-Life: Opposing Force,0.0,0.0
...,...,...,...,...,...
5170010,76561198329548331,373330,All Is Dust,0.0,0.0
5170011,76561198329548331,388490,One Way To Die: Steam Edition,3.0,3.0
5170012,76561198329548331,521570,You Have 10 Seconds 2,4.0,4.0
5170013,76561198329548331,519140,Minds Eyes,3.0,3.0


In [ ]:
# Utilizar value_counts() en la columna 'user_id'
conteo_user_id = df_final['user_id'].value_counts()

# Acceder al conteo del usuario específico (76561197970982479)
conteo_usuario_especifico = conteo_user_id.get('jorellpogi', 0)

# Mostrar el conteo del usuario específico
print("El usuario jorellpogi aparece", conteo_usuario_especifico, "veces en el DataFrame.")

El usuario jorellpogi aparece 71 veces en el DataFrame.


In [ ]:
# Eliminar la columna 'playtime_2weeks' del DataFrame df_final
df_final = df_final.drop(columns=['playtime_2weeks'])

In [ ]:
# Encontrar los valores NaN en el DataFrame
valores_nan = df_final[df_final.isna().any(axis=1)]

# Mostrar los valores NaN
valores_nan   

,user_id,item_id,item_name,playtime_forever
3733,Wackky,NaN,NaN,NaN
3849,76561198079601835,NaN,NaN,NaN
6019,hellom8o,NaN,NaN,NaN
6523,starkillershadow553,NaN,NaN,NaN
7237,darkenkane,NaN,NaN,NaN
...,...,...,...,...
5169470,76561198316380182,NaN,NaN,NaN
5169471,76561198316970597,NaN,NaN,NaN
5169472,76561198318100691,NaN,NaN,NaN
5170006,XxLaughingJackClown77xX,NaN,NaN,NaN


Tenemos 16806 valores con NaN en todos sus campos

In [ ]:
# Suponiendo que tienes un DataFrame llamado games

# Filtrar el DataFrame games para obtener las filas donde item_id sea igual a 249130
item_id = games[games['item_id'] == 1]

# Mostrar las filas relacionadas con el item_id 249130
item_id

,genres,item_name,release_date,price,item_id,developer


In [ ]:
# Llenar los valores NaN en la columna 'item_id' con el valor 1
# df_final['item_id'] = df_final['item_id'].fillna(1)

# # Convertir la columna 'item_id' a int64
# df_final['item_id'] = df_final['item_id'].astype('int64')

Codigo que funciono

In [ ]:
# Filtrar las filas en df_final que tienen valores NaN o 1 en la columna 'item_id'
df_final_filtrado = df_final[df_final['item_id'].isna() | (df_final['item_id'] == 1)]

# Filtrar reviews para obtener solo las columnas 'user_id' y 'item_id'
reviews_filtrado = reviews[['user_id', 'item_id']]

# Fusionar los DataFrames filtrados basados en la columna 'user_id'
df_final_actualizado = pd.merge(df_final_filtrado, reviews_filtrado, on='user_id', how='left', suffixes=('_df_final', '_reviews'))

# Actualizar los valores de 'item_id' en df_final
df_final_actualizado['item_id'] = df_final_actualizado['item_id_reviews'].fillna(df_final_actualizado['item_id_df_final'])

# Conservar los valores de 'item_name' y 'playtime_forever' de df_final
df_final_actualizado = df_final_actualizado[['user_id', 'item_id', 'item_name', 'playtime_forever']]

In [ ]:
# Cambiar el tipo de dato de la columna 'item_id' de float a int64
df_final_actualizado['item_id'] = df_final_actualizado['item_id'].astype('Int64')

In [ ]:
df_final_actualizado

,user_id,item_id,item_name,playtime_forever
0,Wackky,249130,NaN,NaN
1,Wackky,207610,NaN,NaN
2,Wackky,550,NaN,NaN
3,Wackky,65800,NaN,NaN
4,Wackky,220,NaN,NaN
...,...,...,...,...
20322,76561198316380182,<NA>,NaN,NaN
20323,76561198316970597,<NA>,NaN,NaN
20324,76561198318100691,<NA>,NaN,NaN
20325,XxLaughingJackClown77xX,<NA>,NaN,NaN


- Eliminar los valores NaN de df_final
- Unir el dataframe df_final y df_final_actualizado
- Eliminar los valores NaN

In [ ]:
# Eliminar filas con valores NaN en las columnas 'item_id', 'item_name' y 'playtime_forever'
df_final = df_final.dropna(subset=['item_id', 'item_name', 'playtime_forever'])

In [ ]:
df_final

,user_id,item_id,item_name,playtime_forever
0,76561197970982479,10,Counter-Strike,6.0
1,76561197970982479,20,Team Fortress Classic,0.0
2,76561197970982479,30,Day of Defeat,7.0
3,76561197970982479,40,Deathmatch Classic,0.0
4,76561197970982479,50,Half-Life: Opposing Force,0.0
...,...,...,...,...
5170009,76561198329548331,346330,BrainBread 2,0.0
5170010,76561198329548331,373330,All Is Dust,0.0
5170011,76561198329548331,388490,One Way To Die: Steam Edition,3.0
5170012,76561198329548331,521570,You Have 10 Seconds 2,4.0


In [ ]:
# Encontrar los valores NaN en el DataFrame
valores_nan = df_final[df_final.isna().any(axis=1)]

# Mostrar los valores NaN
valores_nan

,user_id,item_id,item_name,playtime_forever


In [ ]:
df_final.shape

(5153209, 4)

Unir dataframes

In [ ]:
# Añadir las filas de df_final_actualizado a df_final
df_final = df_final.append(df_final_actualizado, ignore_index=True)

<ipython-input-22-a813d6c94f4e>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_final.append(df_final_actualizado, ignore_index=True)


In [ ]:
df_final

,user_id,item_id,item_name,playtime_forever
0,76561197970982479,10,Counter-Strike,6.0
1,76561197970982479,20,Team Fortress Classic,0.0
2,76561197970982479,30,Day of Defeat,7.0
3,76561197970982479,40,Deathmatch Classic,0.0
4,76561197970982479,50,Half-Life: Opposing Force,0.0
...,...,...,...,...
5173531,76561198316380182,NaN,NaN,NaN
5173532,76561198316970597,NaN,NaN,NaN
5173533,76561198318100691,NaN,NaN,NaN
5173534,XxLaughingJackClown77xX,NaN,NaN,NaN


user_id, item_id, item_name, playtime_forever, genres, release_date, price, recommend, year, sentiment_analysis


In [ ]:
# Encontrar los valores NaN en el DataFrame
valores_nan = df_final[df_final.isna().any(axis=1)]

# Mostrar los valores NaN
valores_nan

,user_id,item_id,item_name,playtime_forever
5153209,Wackky,249130,NaN,NaN
5153210,Wackky,207610,NaN,NaN
5153211,Wackky,550,NaN,NaN
5153212,Wackky,65800,NaN,NaN
5153213,Wackky,220,NaN,NaN
...,...,...,...,...
5173531,76561198316380182,<NA>,NaN,NaN
5173532,76561198316970597,<NA>,NaN,NaN
5173533,76561198318100691,<NA>,NaN,NaN
5173534,XxLaughingJackClown77xX,<NA>,NaN,NaN


In [ ]:
df_final.isna().sum()

user_id                 0
item_id             13934
item_name           20327
playtime_forever    20327
dtype: int64

In [ ]:
# Filtrar el DataFrame para encontrar las filas donde el usuario_id sea 'Wackky'
df_final[df_final['user_id'] == 'starkillershadow553']

,user_id,item_id,item_name,playtime_forever
5153216,starkillershadow553,238090,NaN,NaN
5153217,starkillershadow553,242130,NaN,NaN
5153218,starkillershadow553,277430,NaN,NaN


In [ ]:
df_final.shape

(5173536, 4)

In [ ]:
# Utilizar value_counts() en la columna 'user_id'
conteo_user_id = df_final['user_id'].value_counts()

# Acceder al conteo del usuario específico (76561197970982479)
conteo_usuario_especifico = conteo_user_id.get('Wackky', 0)

# Mostrar el conteo del usuario específico
print("El usuario aparece", conteo_usuario_especifico, "veces en el DataFrame.")

El usuario aparece 5 veces en el DataFrame.


In [ ]:
df_final

,user_id,item_id,item_name,playtime_forever
0,76561197970982479,10,Counter-Strike,6.0
1,76561197970982479,20,Team Fortress Classic,0.0
2,76561197970982479,30,Day of Defeat,7.0
3,76561197970982479,40,Deathmatch Classic,0.0
4,76561197970982479,50,Half-Life: Opposing Force,0.0
...,...,...,...,...
5173531,76561198316380182,<NA>,NaN,NaN
5173532,76561198316970597,<NA>,NaN,NaN
5173533,76561198318100691,<NA>,NaN,NaN
5173534,XxLaughingJackClown77xX,<NA>,NaN,NaN


In [ ]:
df_final.shape

(5173536, 4)

Cambiar nombres según los **item_name** y **item_id**

In [ ]:
games.head(5)

,genres,item_name,release_date,price,item_id,developer
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018,4.99,761140,Kotoshiro
1,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018,0.00,643980,Secret Level SRL
2,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017,0.00,670290,Poolians.com
3,"[Action, Adventure, Casual]",弹炸人2222,2017,0.99,767400,彼岸领域
4,"[Action, Indie, Casual, Sports]",Log Challenge,0,2.99,773570,Dev-desconocidos


Ahora a eliminar los items que no aportan información al dataframe que tienen valores **`NaN`**

Reemplazar NaN por item_name del df **games**

In [ ]:
# Fusionar los DataFrames df_final y games basados en la columna 'item_id'
df_final = pd.merge(df_final, games[['item_id', 'item_name']], on='item_id', how='left', suffixes=('_df_final', '_games'))

# Reemplazar los valores NaN en la columna 'item_name' de df_final con los valores de games
df_final['item_name_df_final'] = df_final['item_name_df_final'].fillna(df_final['item_name_games'])

# Eliminar las columnas 'item_name_games' ya no necesarias
df_final.drop(columns=['item_name_games'], inplace=True)

In [ ]:
# Renombrar la columna 'item_name_df_final' a 'item_name'
df_final.rename(columns={'item_name_df_final': 'item_name'}, inplace=True)

In [ ]:
df_final

,user_id,item_id,item_name,playtime_forever
0,76561197970982479,10,Counter-Strike,6.0
1,76561197970982479,20,Team Fortress Classic,0.0
2,76561197970982479,30,Day of Defeat,7.0
3,76561197970982479,40,Deathmatch Classic,0.0
4,76561197970982479,50,Half-Life: Opposing Force,0.0
...,...,...,...,...
5173543,76561198316380182,NaN,NaN,NaN
5173544,76561198316970597,NaN,NaN,NaN
5173545,76561198318100691,NaN,NaN,NaN
5173546,XxLaughingJackClown77xX,NaN,NaN,NaN


In [ ]:
# Encontrar los valores NaN en el DataFrame
valores_nan = df_final[df_final.isna().any(axis=1)]

# Mostrar los valores NaN
valores_nan

,user_id,item_id,item_name,playtime_forever
5153209,Wackky,249130,LEGO® Marvel™ Super Heroes,NaN
5153210,Wackky,207610,The Walking Dead,NaN
5153211,Wackky,550,Left 4 Dead 2,NaN
5153212,Wackky,65800,Dungeon Defenders,NaN
5153213,Wackky,220,Half-Life 2,NaN
...,...,...,...,...
5173543,76561198316380182,<NA>,NaN,NaN
5173544,76561198316970597,<NA>,NaN,NaN
5173545,76561198318100691,<NA>,NaN,NaN
5173546,XxLaughingJackClown77xX,<NA>,NaN,NaN


In [ ]:
# Encontrar los valores NaN en el DataFrame
valores_nan = df_final[df_final.isna().any(axis=1)]

# Mostrar los valores NaN
valores_nan

,user_id,item_id,item_name,playtime_forever
5153209,Wackky,249130,LEGO® Marvel™ Super Heroes,NaN
5153210,Wackky,207610,The Walking Dead,NaN
5153211,Wackky,550,Left 4 Dead 2,NaN
5153212,Wackky,65800,Dungeon Defenders,NaN
5153213,Wackky,220,Half-Life 2,NaN
...,...,...,...,...
5173543,76561198316380182,<NA>,NaN,NaN
5173544,76561198316970597,<NA>,NaN,NaN
5173545,76561198318100691,<NA>,NaN,NaN
5173546,XxLaughingJackClown77xX,<NA>,NaN,NaN


Reemplazar valores NaN de la columna item_name por Desconocido

In [ ]:
# Filtrar las filas donde item_name es NaN y item_id no es NaN
nan_item_name = df_final.loc[df_final['item_name'].isna() & df_final['item_id'].notna()]

# Obtener los item_id únicos que cumplen la condición
item_ids_with_nan_name = nan_item_name['item_id'].unique()

# Iterar sobre los item_id únicos y reemplazar sus item_name por "Desconocido"
for item_id in item_ids_with_nan_name:
    df_final.loc[(df_final['item_id'] == item_id), 'item_name'] = df_final.loc[(df_final['item_id'] == item_id), 'item_name'].fillna("Desconocido")

In [ ]:
# Filtrar las filas donde playtime_forever es NaN y item_id no es NaN
mask = df_final['playtime_forever'].isna() & df_final['item_id'].notna()

# Reemplazar los valores NaN en playtime_forever por 0 donde item_id no es NaN
df_final.loc[mask, 'playtime_forever'] = 0

In [ ]:
# Encontrar los valores NaN en el DataFrame
valores_nan = df_final[df_final.isna().any(axis=1)]

# Mostrar los valores NaN
valores_nan

,user_id,item_id,item_name,playtime_forever
5153215,hellom8o,<NA>,NaN,NaN
5153219,darkenkane,<NA>,NaN,NaN
5153220,Bubblecup,<NA>,NaN,NaN
5153221,76561198082525427,<NA>,NaN,NaN
5153222,76561198048504366,<NA>,NaN,NaN
...,...,...,...,...
5173543,76561198316380182,<NA>,NaN,NaN
5173544,76561198316970597,<NA>,NaN,NaN
5173545,76561198318100691,<NA>,NaN,NaN
5173546,XxLaughingJackClown77xX,<NA>,NaN,NaN


Eliminamos los valores NaN de las filas que no aportan información, son 13934 registros

In [ ]:
# Eliminar filas con valores NaN en las columnas 'item_id', 'item_name' y 'playtime_forever'
df_final = df_final.dropna(subset=['item_id', 'item_name', 'playtime_forever'])

In [ ]:
# Encontrar los valores NaN en el DataFrame
valores_nan = df_final[df_final.isna().any(axis=1)]

# Mostrar los valores NaN
valores_nan

,user_id,item_id,item_name,playtime_forever


In [ ]:
df_final.shape

(5159614, 4)

In [ ]:
df_final.isna().sum()

user_id             0
item_id             0
item_name           0
playtime_forever    0
dtype: int64

In [ ]:
df_final_actualizado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20327 entries, 0 to 20326
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user_id           20327 non-null  object 
 1   item_id           6393 non-null   Int64  
 2   item_name         0 non-null      object 
 3   playtime_forever  0 non-null      float64
dtypes: Int64(1), float64(1), object(2)
memory usage: 813.9+ KB


In [ ]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5159614 entries, 0 to 5173541
Data columns (total 4 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   item_id           object 
 2   item_name         object 
 3   playtime_forever  float64
dtypes: float64(1), object(3)
memory usage: 196.8+ MB


In [ ]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        32135 non-null  object 
 1   item_name     32135 non-null  object 
 2   release_date  32135 non-null  int64  
 3   price         32135 non-null  float64
 4   item_id       32135 non-null  int64  
 5   developer     32135 non-null  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 1.5+ MB


In [ ]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   item_id             59305 non-null  int64 
 1   recommend           59305 non-null  bool  
 2   year                59305 non-null  int64 
 3   sentiment_analysis  59305 non-null  int64 
 4   user_id             59305 non-null  object
dtypes: bool(1), int64(3), object(1)
memory usage: 1.9+ MB


In [ ]:
# prompt: convertir la columna item_id a entero

df_final['item_id'] = df_final['item_id'].astype('int64')


<ipython-input-48-437f2c234d29>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['item_id'] = df_final['item_id'].astype('int64')


### Ahora unimos las columnas de los otros dataframes **Reviews** y **Games**

In [ ]:
df_final

,user_id,item_id,item_name,playtime_forever
0,76561197970982479,10,Counter-Strike,6.0
1,76561197970982479,20,Team Fortress Classic,0.0
2,76561197970982479,30,Day of Defeat,7.0
3,76561197970982479,40,Deathmatch Classic,0.0
4,76561197970982479,50,Half-Life: Opposing Force,0.0
...,...,...,...,...
5173481,Fuckfhaisjnsnsjakaka,730,Counter-Strike: Global Offensive,0.0
5173498,KinkyyyCSGO,282660,EasyAntiCheat eSports,0.0
5173539,LydiaMorley,273110,Counter-Strike Nexon: Zombies,0.0
5173540,LydiaMorley,730,Counter-Strike: Global Offensive,0.0


In [ ]:
reviews

,item_id,recommend,year,sentiment_analysis,user_id
0,1250,True,2011,2,76561197970982479
1,22200,True,2011,2,76561197970982479
2,43110,True,2011,2,76561197970982479
3,251610,True,2014,2,js41637
4,227300,True,2013,2,js41637
...,...,...,...,...,...
59300,70,True,0,2,76561198312638244
59301,362890,True,0,2,76561198312638244
59302,273110,True,0,2,LydiaMorley
59303,730,True,0,2,LydiaMorley


In [ ]:
# Fusionar los DataFrames df_final y reviews basados en las columnas 'user_id' y 'item_id'
df_final = df_final.merge(reviews[['user_id', 'item_id', 'recommend', 'year', 'sentiment_analysis']], on=['user_id', 'item_id'], how='left')

In [ ]:
df_final

/usr/local/lib/python3.10/dist-packages/IPython/lib/pretty.py:700: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  output = repr(obj)
/usr/local/lib/python3.10/dist-packages/google/colab/_interactive_table_hint_button.py:178: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df_html=dataframe._repr_html_(),  # pylint: disable=protected-access


,user_id,item_id,item_name,playtime_forever,recommend,year,sentiment_analysis
0,76561197970982479,10,Counter-Strike,6.0,NaN,NaN,NaN
1,76561197970982479,20,Team Fortress Classic,0.0,NaN,NaN,NaN
2,76561197970982479,30,Day of Defeat,7.0,NaN,NaN,NaN
3,76561197970982479,40,Deathmatch Classic,0.0,NaN,NaN,NaN
4,76561197970982479,50,Half-Life: Opposing Force,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...
5161329,Fuckfhaisjnsnsjakaka,730,Counter-Strike: Global Offensive,0.0,True,0.0,2.0
5161330,KinkyyyCSGO,282660,EasyAntiCheat eSports,0.0,True,0.0,2.0
5161331,LydiaMorley,273110,Counter-Strike Nexon: Zombies,0.0,True,0.0,2.0
5161332,LydiaMorley,730,Counter-Strike: Global Offensive,0.0,True,0.0,2.0


In [ ]:
# prompt: valores unicos de sentiment_analysis

df_final['sentiment_analysis'].unique()


array([nan,  2.,  1.,  0.])

In [ ]:
# Aplicar la asignación condicional solo a los valores NaN en 'recommend'
df_final['recommend'].fillna(df_final['playtime_forever'] > 0, inplace=True)

In [ ]:
df_final

,user_id,item_id,item_name,playtime_forever,recommend,year,sentiment_analysis
0,76561197970982479,10,Counter-Strike,6.0,True,NaN,NaN
1,76561197970982479,20,Team Fortress Classic,0.0,False,NaN,NaN
2,76561197970982479,30,Day of Defeat,7.0,True,NaN,NaN
3,76561197970982479,40,Deathmatch Classic,0.0,False,NaN,NaN
4,76561197970982479,50,Half-Life: Opposing Force,0.0,False,NaN,NaN
...,...,...,...,...,...,...,...
5161329,Fuckfhaisjnsnsjakaka,730,Counter-Strike: Global Offensive,0.0,True,0.0,2.0
5161330,KinkyyyCSGO,282660,EasyAntiCheat eSports,0.0,True,0.0,2.0
5161331,LydiaMorley,273110,Counter-Strike Nexon: Zombies,0.0,True,0.0,2.0
5161332,LydiaMorley,730,Counter-Strike: Global Offensive,0.0,True,0.0,2.0


In [ ]:
# Reemplazar los valores NaN en la columna 'year' por 0
df_final['year'].fillna(0, inplace=True)

In [ ]:
# prompt: valores unicos de sentiment_analysis

df_final['sentiment_analysis'].unique()

array([nan,  2.,  1.,  0.])

In [ ]:
# Aplicar la lógica de imputación a la columna 'sentiment_analysis' solo para valores NaN
mask = df_final['sentiment_analysis'].isnull()
df_final.loc[mask, 'sentiment_analysis'] = np.where(df_final.loc[mask, 'playtime_forever'] > 0, 2, 1)

In [ ]:
# prompt: valores unicos de sentiment_analysis

df_final['sentiment_analysis'].unique()

array([2., 1., 0.])

In [ ]:
# Contar la cantidad de cada valor en la columna 'review'
df_final['sentiment_analysis'].value_counts()

2.0    3274336
1.0    1877978
0.0       9020
Name: sentiment_analysis, dtype: int64

In [ ]:
# Crear un diccionario de mapeo de item_id a year
mapeo_year = dict(df_final[df_final['year'] != 0][['item_id', 'year']].values)

# Imputar los valores de 'year' basados en el mapeo
df_final['year'] = df_final['year'].fillna(df_final['item_id'].map(mapeo_year))

In [ ]:
# Seleccionar las columnas que se agregarán a df_final
columns_to_add = ['item_id', 'genres', 'release_date', 'price', 'developer']

# Fusionar los DataFrames basados en la columna 'item_id'
df_final = pd.merge(df_final, games[columns_to_add], on='item_id', how='left')

In [ ]:
df_final

,user_id,item_id,item_name,playtime_forever,recommend,year,sentiment_analysis,genres,release_date,price,developer
0,76561197970982479,10,Counter-Strike,6.0,True,0.0,2.0,[Action],2000.0,9.99,Valve
1,76561197970982479,20,Team Fortress Classic,0.0,False,0.0,1.0,[Action],1999.0,4.99,Valve
2,76561197970982479,30,Day of Defeat,7.0,True,0.0,2.0,[Action],2003.0,4.99,Valve
3,76561197970982479,40,Deathmatch Classic,0.0,False,0.0,1.0,[Action],2001.0,4.99,Valve
4,76561197970982479,50,Half-Life: Opposing Force,0.0,False,0.0,1.0,[Action],1999.0,4.99,Gearbox Software
...,...,...,...,...,...,...,...,...,...,...,...
5170016,Fuckfhaisjnsnsjakaka,730,Counter-Strike: Global Offensive,0.0,True,0.0,2.0,[Action],2012.0,14.99,Valve
5170017,KinkyyyCSGO,282660,EasyAntiCheat eSports,0.0,True,0.0,2.0,[Utilities],2006.0,0.00,EasyAntiCheat Ltd
5170018,LydiaMorley,273110,Counter-Strike Nexon: Zombies,0.0,True,0.0,2.0,"[Action, Casual, Free to Play, Massively Multi...",2014.0,0.00,Nexon
5170019,LydiaMorley,730,Counter-Strike: Global Offensive,0.0,True,0.0,2.0,[Action],2012.0,14.99,Valve


In [ ]:
# prompt: noto que genres esta en array quitar ese array y convertirlo en una lista {'user_id': 'js41637',   'item_id': 346010,   'item_name': 'Besiege',   'playtime_forever': 51,   'recommend': True,   'year': 0.0,   'sentiment_analysis': 2,   'genres': array(['Indie', 'Simulation', 'Early Access'], dtype=object),   'release_date': 2015.0,   'price': 7.99,   'developer': 'Spiderling Studios'},

df_final['genres'] = df_final['genres'].apply(lambda x: x.tolist() if isinstance(x, np.ndarray) else x)

In [ ]:
unique_genres = set()

# Iterar sobre cada lista en la columna 'genres'
for genres_list in df_final['genres']:
    # Verificar si la lista es NaN
    if isinstance(genres_list, list):
        # Agregar los elementos de la lista al conjunto unique_genres
        unique_genres.update(genres_list)
    else:
        # Si es NaN, agregar un valor especial al conjunto
        unique_genres.add('NaN')

# Imprimir los géneros únicos, incluido 'NaN'
print(unique_genres)

{'Survival Horror', 'Design & Illustration', 'Funny', 'Dragons', 'Cartoony', 'Software Training', 'Cinematic', 'Romance', 'Multiplayer', 'Colorful', 'Survival', 'On-Rails Shooter', 'Time Attack', 'Golf', 'Card Game', 'Level Editor', 'Hardware', 'Platformer', 'Puzzle-Platformer', 'Sandbox', 'Split Screen', 'Comedy', 'Third-Person Shooter', "Shoot 'Em Up", 'World War I', 'NaN', 'Fishing', 'Massively Multiplayer', 'Classic', 'Driving', 'Tanks', 'Horses', 'Difficult', 'PvE', 'Aliens', 'Minimalist', 'Arcade', 'Runner', 'MOBA', 'Science', 'Football', 'Exploration', 'Offroad', 'Fast-Paced', 'Cartoon', 'Twin Stick Shooter', 'Steam Machine', 'Sci-fi', 'Comic Book', 'Mini Golf', 'Psychological Horror', 'Hacking', 'Gaming', 'Cute', 'Memes', 'Physics', 'Dynamic Narration', 'Nonlinear', 'Abstract', 'Futuristic', 'Rogue-lite', 'Multiple Endings', 'Top-Down', 'Space Sim', '2D', 'Steampunk', 'Music', 'TrackIR', 'Replay Value', 'Historical', 'Game Development', 'Pirates', 'PvP', 'VR', 'Thriller', 'Park

In [ ]:
# Reemplazar los valores NaN en la columna 'genres' por una lista de un solo elemento que contiene 'Desconocido'
df_final['genres'] = df_final['genres'].apply(lambda x: ['Desconocido'] if isinstance(x, float) else x)

In [ ]:
# Contar la frecuencia de 'Desconocido' en la columna 'genres'
frecuencia_desconocido = df_final['genres'].apply(lambda x: 'Desconocido' in x).sum()

print("Frecuencia de 'Desconocido' en la columna 'genres':", frecuencia_desconocido)

Frecuencia de 'Desconocido' en la columna 'genres': 859787


In [ ]:
# prompt: cambiar el tipo de dato de la columna playtime_forever  a int

df_final['playtime_forever'] = df_final['playtime_forever'].astype('int64')


In [ ]:
def UserForGenre(genero: str):
    # Filtrar el DataFrame por el género dado
    df_genre = df_final[df_final['genres'].apply(lambda x: genero in x)]

    # Si el DataFrame está vacío, lanzar una excepción
    if df_genre.empty:
        return {"detail": [{"loc": ["string", 0], "msg": "El género no existe", "type": "value_error"}]}

    # Filtrar los géneros según los registros de cada user_id y release_date
    df_filtered = df_genre.groupby(['user_id', 'release_date'])['playtime_forever'].sum().reset_index()

    # Obtener el usuario con más horas jugadas para el género dado
    user_most_played_id = df_filtered.groupby('user_id')['playtime_forever'].sum().idxmax()

    # Obtener la acumulación de horas jugadas por año de lanzamiento
    hours_per_year = get_hours_per_year(df_filtered)

    # Devolver los resultados en el formato especificado
    return {"Usuario con más horas jugadas para el género {}".format(genero): user_most_played_id, "Horas jugadas": hours_per_year}

def get_hours_per_year(df_genre):
    # Agrupar por año de lanzamiento y sumar las horas jugadas
    hours_per_year = df_genre.groupby('release_date')['playtime_forever'].sum().astype(int).reset_index()
    hours_per_year.columns = ['Año', 'Horas']

    # Cambiar el valor del año a 'desconocido' cuando es igual a 0
    hours_per_year['Año'] = hours_per_year['Año'].apply(lambda x: 'Desconocido' if x == 0 else x)

    # Crear la lista de resultados
    hours_per_year = hours_per_year.to_dict('records')

    return hours_per_year

In [ ]:
df_final

,user_id,item_id,item_name,playtime_forever,recommend,year,sentiment_analysis,genres,release_date,price,developer
0,76561197970982479,10,Counter-Strike,6,True,0.0,2.0,[Action],2000.0,9.99,Valve
1,76561197970982479,20,Team Fortress Classic,0,False,0.0,1.0,[Action],1999.0,4.99,Valve
2,76561197970982479,30,Day of Defeat,7,True,0.0,2.0,[Action],2003.0,4.99,Valve
3,76561197970982479,40,Deathmatch Classic,0,False,0.0,1.0,[Action],2001.0,4.99,Valve
4,76561197970982479,50,Half-Life: Opposing Force,0,False,0.0,1.0,[Action],1999.0,4.99,Gearbox Software
...,...,...,...,...,...,...,...,...,...,...,...
5170016,Fuckfhaisjnsnsjakaka,730,Counter-Strike: Global Offensive,0,True,0.0,2.0,[Action],2012.0,14.99,Valve
5170017,KinkyyyCSGO,282660,EasyAntiCheat eSports,0,True,0.0,2.0,[Utilities],2006.0,0.00,EasyAntiCheat Ltd
5170018,LydiaMorley,273110,Counter-Strike Nexon: Zombies,0,True,0.0,2.0,"[Action, Casual, Free to Play, Massively Multi...",2014.0,0.00,Nexon
5170019,LydiaMorley,730,Counter-Strike: Global Offensive,0,True,0.0,2.0,[Action],2012.0,14.99,Valve


In [ ]:
# prompt: llamar a la funcion UserForGenre
UserForGenre('Narration')


{'Usuario con más horas jugadas para el género Narration': 'baeritto',
 'Horas jugadas': [{'Año': 'Desconocido', 'Horas': 4507},
  {'Año': 2016.0, 'Horas': 195}]}

In [ ]:
# prompt: buscar al usuario 76561198019112245

# Filtrar el DataFrame para encontrar las filas donde el usuario_id sea '76561198019112245'
df_final[df_final['user_id'] == 'Wackky']


,user_id,item_id,item_name,playtime_forever,recommend,year,sentiment_analysis,genres,release_date,price,developer
5163296,Wackky,249130,LEGO® Marvel™ Super Heroes,0,True,2014.0,1.0,"[Action, Adventure]",2013.0,19.99,Traveller's Tales
5163297,Wackky,207610,The Walking Dead,0,True,2012.0,1.0,[Adventure],2012.0,24.99,Telltale Games
5163298,Wackky,550,Left 4 Dead 2,0,True,2012.0,2.0,[Action],2009.0,19.99,Valve
5163299,Wackky,65800,Dungeon Defenders,0,True,2012.0,2.0,"[Action, Indie, RPG, Strategy]",2011.0,14.99,Trendy Entertainment
5163300,Wackky,220,Half-Life 2,0,True,2012.0,2.0,[Action],2004.0,9.99,Valve


In [ ]:
unique_genres = set()

# Iterar sobre cada lista en la columna 'genres'
for genres_list in df_final['genres']:
    # Verificar si la lista es NaN
    if isinstance(genres_list, list):
        # Agregar los elementos de la lista al conjunto unique_genres
        unique_genres.update(genres_list)
    else:
        # Si es NaN, agregar un valor especial al conjunto
        unique_genres.add('NaN')

# Imprimir los géneros únicos, incluido 'NaN'
print(unique_genres)


{'Survival Horror', 'Design & Illustration', 'Funny', 'Dragons', 'Cartoony', 'Software Training', 'Cinematic', 'Romance', 'Multiplayer', 'Colorful', 'Survival', 'On-Rails Shooter', 'Time Attack', 'Golf', 'Card Game', 'Level Editor', 'Hardware', 'Platformer', 'Puzzle-Platformer', 'Sandbox', 'Split Screen', 'Comedy', 'Third-Person Shooter', "Shoot 'Em Up", 'World War I', 'Fishing', 'Massively Multiplayer', 'Classic', 'Driving', 'Tanks', 'Horses', 'Difficult', 'PvE', 'Aliens', 'Minimalist', 'Arcade', 'Runner', 'MOBA', 'Science', 'Football', 'Exploration', 'Offroad', 'Fast-Paced', 'Cartoon', 'Twin Stick Shooter', 'Steam Machine', 'Sci-fi', 'Comic Book', 'Mini Golf', 'Psychological Horror', 'Hacking', 'Gaming', 'Cute', 'Memes', 'Physics', 'Dynamic Narration', 'Nonlinear', 'Abstract', 'Futuristic', 'Rogue-lite', 'Multiple Endings', 'Top-Down', 'Space Sim', '2D', 'Steampunk', 'Music', 'TrackIR', 'Replay Value', 'Historical', 'Game Development', 'Pirates', 'PvP', 'VR', 'Thriller', 'Parkour', '

In [ ]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5170021 entries, 0 to 5170020
Data columns (total 11 columns):
 #   Column              Dtype  
---  ------              -----  
 0   user_id             object 
 1   item_id             int64  
 2   item_name           object 
 3   playtime_forever    int64  
 4   recommend           bool   
 5   year                float64
 6   sentiment_analysis  float64
 7   genres              object 
 8   release_date        float64
 9   price               float64
 10  developer           object 
dtypes: bool(1), float64(4), int64(2), object(4)
memory usage: 438.8+ MB


In [ ]:


df_final[df_final['user_id'] == 'Sp3ctre']


,user_id,item_id,item_name,playtime_forever,recommend,year,sentiment_analysis,genres,release_date,price,developer
1597180,Sp3ctre,300,Day of Defeat: Source,249,True,0.0,2.0,[Action],2010.0,9.99,Valve
1597181,Sp3ctre,240,Counter-Strike: Source,122426,True,0.0,2.0,[Action],2004.0,19.99,Valve
1597182,Sp3ctre,4000,Garry's Mod,89357,True,0.0,2.0,"[Indie, Simulation]",2006.0,9.99,Facepunch Studios
1597183,Sp3ctre,2620,Call of Duty,7673,True,0.0,2.0,[Action],2003.0,19.99,Infinity Ward
1597184,Sp3ctre,2630,Call of Duty 2,21339,True,0.0,2.0,[Action],2005.0,19.99,"Infinity Ward,Aspyr (Mac)"
...,...,...,...,...,...,...,...,...,...,...,...
1597747,Sp3ctre,273350,Evolve Stage 2,0,False,0.0,1.0,"[Action, Free to Play]",2015.0,0.00,Turtle Rock Studios
1597748,Sp3ctre,504490,The God's Chain,0,False,0.0,1.0,[Action],2016.0,7.99,CRASS Studios
1597749,Sp3ctre,504980,Elves Adventure,0,False,0.0,1.0,"[Action, Adventure]",2016.0,1.99,Patryk Taranek
1597750,Sp3ctre,510050,You Have 10 Seconds,0,False,0.0,1.0,"[Free to Play, Indie]",2016.0,0.00,tamationgames


In [ ]:

df_final[(df_final['user_id'] == 'Sp3ctre') & (df_final['release_date'] == 1983)]


,user_id,item_id,item_name,playtime_forever,recommend,year,sentiment_analysis,genres,release_date,price,developer


In [ ]:
df_final

,user_id,item_id,item_name,playtime_forever,recommend,year,sentiment_analysis,genres,release_date,price,developer
0,76561197970982479,10,Counter-Strike,6,True,0.0,2.0,[Action],2000.0,9.99,Valve
1,76561197970982479,20,Team Fortress Classic,0,False,0.0,1.0,[Action],1999.0,4.99,Valve
2,76561197970982479,30,Day of Defeat,7,True,0.0,2.0,[Action],2003.0,4.99,Valve
3,76561197970982479,40,Deathmatch Classic,0,False,0.0,1.0,[Action],2001.0,4.99,Valve
4,76561197970982479,50,Half-Life: Opposing Force,0,False,0.0,1.0,[Action],1999.0,4.99,Gearbox Software
...,...,...,...,...,...,...,...,...,...,...,...
5170016,Fuckfhaisjnsnsjakaka,730,Counter-Strike: Global Offensive,0,True,0.0,2.0,[Action],2012.0,14.99,Valve
5170017,KinkyyyCSGO,282660,EasyAntiCheat eSports,0,True,0.0,2.0,[Utilities],2006.0,0.00,EasyAntiCheat Ltd
5170018,LydiaMorley,273110,Counter-Strike Nexon: Zombies,0,True,0.0,2.0,"[Action, Casual, Free to Play, Massively Multi...",2014.0,0.00,Nexon
5170019,LydiaMorley,730,Counter-Strike: Global Offensive,0,True,0.0,2.0,[Action],2012.0,14.99,Valve


In [ ]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5170021 entries, 0 to 5170020
Data columns (total 11 columns):
 #   Column              Dtype  
---  ------              -----  
 0   user_id             object 
 1   item_id             int64  
 2   item_name           object 
 3   playtime_forever    int64  
 4   recommend           bool   
 5   year                float64
 6   sentiment_analysis  float64
 7   genres              object 
 8   release_date        float64
 9   price               float64
 10  developer           object 
dtypes: bool(1), float64(4), int64(2), object(4)
memory usage: 438.8+ MB


In [ ]:
# Paso 1: Crear un diccionario de mapeo de item_id a release_date
id_to_release = dict(df_final[df_final['release_date'].notnull()][['item_id', 'release_date']].values)

# Paso 2: Actualizar los valores NaN en la columna release_date
df_final['release_date'] = df_final.apply(lambda row: id_to_release.get(row['item_id'], row['release_date']), axis=1)

In [ ]:
# Reemplazar los valores NaN en la columna release_date por 0
df_final['release_date'].fillna(0, inplace=True)

In [ ]:
# Reemplazar los valores NaN en la columna price por 0
df_final['price'].fillna(0, inplace=True)

In [ ]:
# Identificar los desarrolladores únicos para cada item_id
developer_mapping = df_final.groupby('item_id')['developer'].first().to_dict()

# Reemplazar los valores NaN en la columna developer basados en el mapeo
df_final['developer'] = df_final.apply(lambda row: developer_mapping[row['item_id']] if pd.isna(row['developer']) else row['developer'], axis=1)

In [ ]:
# Reemplazar los valores NaN en la columna developer por "Dev-desconocidos"
df_final['developer'].fillna('Dev-desconocidos', inplace=True)

In [ ]:
# Encontrar los valores NaN en el DataFrame
valores_nan = df_final[df_final.isna().any(axis=1)]

# Mostrar los valores NaN
valores_nan

,user_id,item_id,item_name,playtime_forever,recommend,year,sentiment_analysis,genres,release_date,price,developer


In [ ]:
# prompt: ver valor de developer Dev-Desconocidos

df_final[df_final['developer'] == 'Dev-desconocidos']


,user_id,item_id,item_name,playtime_forever,recommend,year,sentiment_analysis,genres,release_date,price,developer
13,76561197970982479,34440,Sid Meier's Civilization IV,0,False,0.0,1.0,[Desconocido],0.0,0.0,Dev-desconocidos
26,76561197970982479,9340,Company of Heroes: Opposing Fronts,692,True,0.0,2.0,[Desconocido],0.0,0.0,Dev-desconocidos
27,76561197970982479,228200,Company of Heroes (New Steam Version),0,False,0.0,1.0,[Desconocido],0.0,0.0,Dev-desconocidos
37,76561197970982479,17340,Crysis Wars,0,False,0.0,1.0,[Desconocido],0.0,0.0,Dev-desconocidos
51,76561197970982479,23120,Droplitz,53,True,0.0,2.0,[Desconocido],0.0,0.0,Dev-desconocidos
...,...,...,...,...,...,...,...,...,...,...,...
5169974,BadMadMurphy,271290,Desconocido,0,True,2015.0,2.0,[Desconocido],0.0,0.0,Dev-desconocidos
5169984,cranialperv,203250,Desconocido,0,True,2014.0,2.0,[Desconocido],0.0,0.0,Dev-desconocidos
5169997,76561198209894493,414390,Desconocido,0,True,2015.0,2.0,[Desconocido],0.0,0.0,Dev-desconocidos
5170005,76561198209894493,570,Dota 2,0,False,2015.0,0.0,"[Free to Play, MOBA, Strategy, Multiplayer, Te...",0.0,0.0,Dev-desconocidos


In [ ]:
df_final.isna().sum()

user_id               0
item_id               0
item_name             0
playtime_forever      0
recommend             0
year                  0
sentiment_analysis    0
genres                0
release_date          0
price                 0
developer             0
dtype: int64

In [ ]:

df_final['genres'] = df_final['genres'].astype(str)
df_final['user_id'] = df_final['user_id'].astype(str)
df_final['item_name'] = df_final['item_name'].astype(str)
df_final['year'] = df_final['year'].astype(int)
df_final['release_date'] = df_final['release_date'].astype(int)
df_final['developer'] = df_final['developer'].astype(str)
df_final['sentiment_analysis'] = df_final['sentiment_analysis'].astype(int)


In [ ]:

df_final.drop_duplicates(subset=['user_id', 'item_id', 'item_name', 'playtime_forever', 'recommend', 'year', 'sentiment_analysis', 'genres', 'release_date', 'price', 'developer'], inplace=True)


In [ ]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5100274 entries, 0 to 5170020
Data columns (total 11 columns):
 #   Column              Dtype  
---  ------              -----  
 0   user_id             object 
 1   item_id             int64  
 2   item_name           object 
 3   playtime_forever    int64  
 4   recommend           bool   
 5   year                int64  
 6   sentiment_analysis  int64  
 7   genres              object 
 8   release_date        int64  
 9   price               float64
 10  developer           object 
dtypes: bool(1), float64(1), int64(5), object(4)
memory usage: 432.9+ MB


In [ ]:
df_final

,user_id,item_id,item_name,playtime_forever,recommend,year,sentiment_analysis,genres,release_date,price,developer
0,76561197970982479,10,Counter-Strike,6,True,0,2,['Action'],2000,9.99,Valve
1,76561197970982479,20,Team Fortress Classic,0,False,0,1,['Action'],1999,4.99,Valve
2,76561197970982479,30,Day of Defeat,7,True,0,2,['Action'],2003,4.99,Valve
3,76561197970982479,40,Deathmatch Classic,0,False,0,1,['Action'],2001,4.99,Valve
4,76561197970982479,50,Half-Life: Opposing Force,0,False,0,1,['Action'],1999,4.99,Gearbox Software
...,...,...,...,...,...,...,...,...,...,...,...
5170016,Fuckfhaisjnsnsjakaka,730,Counter-Strike: Global Offensive,0,True,0,2,['Action'],2012,14.99,Valve
5170017,KinkyyyCSGO,282660,EasyAntiCheat eSports,0,True,0,2,['Utilities'],2006,0.00,EasyAntiCheat Ltd
5170018,LydiaMorley,273110,Counter-Strike Nexon: Zombies,0,True,0,2,"['Action', 'Casual', 'Free to Play', 'Massivel...",2014,0.00,Nexon
5170019,LydiaMorley,730,Counter-Strike: Global Offensive,0,True,0,2,['Action'],2012,14.99,Valve


In [ ]:
# Resetear el indice

df_final = df_final.reset_index(drop=True)


In [ ]:
# Convertir df_final en un archivo parquet

df_final.to_parquet('/content/drive/MyDrive/Dict/df_final.parquet')


### Funciones

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# prompt: abrir el archivo parquet df_final

# Abrir el archivo parquet df_final
df_final = pd.read_parquet('/content/drive/MyDrive/Dict/df_final.parquet')

In [ ]:
# df_final['genres'] = df_final['genres'].apply(lambda x: x.tolist() if isinstance(x, np.ndarray) else x)

In [ ]:
# prompt: eliminar valores duplicados si aparece más de una vez y es el mismo user_id, item_id, item_name, playtime_forever, recommend, year, sentiment_analysis, genres, release_date, price, developer

# df_final.drop_duplicates(subset=['user_id', 'item_id', 'item_name', 'playtime_forever', 'recommend', 'year', 'sentiment_analysis', 'genres', 'release_date', 'price', 'developer'], inplace=True)


In [ ]:
def UserForGenre(genero: str):
    # Filtrar el DataFrame por el género dado
    df_genre = df_final[df_final['genres'].apply(lambda x: genero in x)]

    # Si el DataFrame está vacío, lanzar una excepción
    if df_genre.empty:
        return {"detail": [{"loc": ["string", 0], "msg": "El género no existe", "type": "value_error"}]}

    # Obtener el usuario que acumula más horas jugadas para el género dado
    user_most_played_id = df_genre.groupby('user_id')['playtime_forever'].sum().idxmax()

    # Filtrar el DataFrame por el usuario con más horas jugadas para el género dado
    df_user_genre = df_genre[df_genre['user_id'] == user_most_played_id]

    # Obtener la acumulación de horas jugadas por año de lanzamiento
    hours_per_year = get_hours_per_year(df_user_genre)

    # Devolver los resultados en el formato especificado
    return {"Usuario con más horas jugadas para el género {}".format(genero): user_most_played_id, "Horas jugadas": hours_per_year}

def get_hours_per_year(df_genre):
    # Agrupar por año de lanzamiento y sumar las horas jugadas
    hours_per_year = df_genre.groupby('release_date')['playtime_forever'].sum().astype(int).reset_index()
    hours_per_year.columns = ['Año', 'Horas']

    # Cambiar el valor del año a 'desconocido' cuando es igual a 0
    hours_per_year['Año'] = hours_per_year['Año'].apply(lambda x: 'Desconocido' if x == 0 else x)

    # Crear la lista de resultados
    hours_per_year = hours_per_year.to_dict('records')

    return hours_per_year

In [ ]:
# prompt: buscar al usuario Sp3ctre

df_final[df_final['user_id'] == 'Sp3ctre']


,user_id,item_id,item_name,playtime_forever,recommend,year,sentiment_analysis,genres,release_date,price,developer
1571889,Sp3ctre,300,Day of Defeat: Source,249,True,0,2,['Action'],2010,9.99,Valve
1571890,Sp3ctre,240,Counter-Strike: Source,122426,True,0,2,['Action'],2004,19.99,Valve
1571891,Sp3ctre,4000,Garry's Mod,89357,True,0,2,"['Indie', 'Simulation']",2006,9.99,Facepunch Studios
1571892,Sp3ctre,2620,Call of Duty,7673,True,0,2,['Action'],2003,19.99,Infinity Ward
1571893,Sp3ctre,2630,Call of Duty 2,21339,True,0,2,['Action'],2005,19.99,"Infinity Ward,Aspyr (Mac)"
...,...,...,...,...,...,...,...,...,...,...,...
1572455,Sp3ctre,273350,Evolve Stage 2,0,False,0,1,"['Action', 'Free to Play']",2015,0.00,Turtle Rock Studios
1572456,Sp3ctre,504490,The God's Chain,0,False,0,1,['Action'],2016,7.99,CRASS Studios
1572457,Sp3ctre,504980,Elves Adventure,0,False,0,1,"['Action', 'Adventure']",2016,1.99,Patryk Taranek
1572458,Sp3ctre,510050,You Have 10 Seconds,0,False,0,1,"['Free to Play', 'Indie']",2016,0.00,tamationgames


In [ ]:
# prompt: llamar a la funcion UserForGenre
UserForGenre('Sexual Content')

{'Usuario con más horas jugadas para el género Sexual Content': 'REBAS_AS_F-T',
 'Horas jugadas': [{'Año': 'Desconocido', 'Horas': 3201}]}

In [ ]:
# prompt: buscar el genres Romance pero en el dataframe sin usar esa funcion

df_final[df_final['genres'].apply(lambda x: 'Anime' in x)]


,user_id,item_id,item_name,playtime_forever,recommend,year,sentiment_analysis,genres,release_date,price,developer
580,js41637,71250,Sonic Adventure DX,0,False,0,1,"['Platformer', 'Adventure', '3D Platformer', '...",2011,7.99,SEGA
1142,js41637,439260,BUTTS,0,False,0,1,"['Psychological Horror', 'Casual', 'VR', 'Meme...",0,0.99,Dev-desconocidos
3342,maplemage,71250,Sonic Adventure DX,0,False,0,1,"['Platformer', 'Adventure', '3D Platformer', '...",2011,7.99,SEGA
3343,maplemage,200940,Sonic CD,106,True,0,2,"['Platformer', 'Classic', 'Great Soundtrack', ...",2012,4.99,Blit Software
4380,76561198156664158,413850,CS:GO Player Profiles,3,True,0,2,"['Free to Play', 'FPS', 'Gaming', 'Shooter', '...",2015,0.00,Dev-desconocidos
...,...,...,...,...,...,...,...,...,...,...,...
5091654,76561198284888779,413850,CS:GO Player Profiles,0,False,0,1,"['Free to Play', 'FPS', 'Gaming', 'Shooter', '...",2015,0.00,Dev-desconocidos
5093886,76561198323066619,413850,CS:GO Player Profiles,0,False,0,1,"['Free to Play', 'FPS', 'Gaming', 'Shooter', '...",2015,0.00,Dev-desconocidos
5095820,GERDAGUFERSD,71250,Sonic Adventure DX,0,True,2015,0,"['Platformer', 'Adventure', '3D Platformer', '...",2011,7.99,SEGA
5097292,killerroo,413850,CS:GO Player Profiles,0,True,2015,2,"['Free to Play', 'FPS', 'Gaming', 'Shooter', '...",2015,0.00,Dev-desconocidos


In [ ]:
df_final[df_final['user_id'] == '76561198019112245']

,user_id,item_id,item_name,playtime_forever,recommend,year,sentiment_analysis,genres,release_date,price,developer
2315215,76561198019112245,240,Counter-Strike: Source,169925,True,0,2,['Action'],2004,19.99,Valve
2315216,76561198019112245,300,Day of Defeat: Source,1,True,0,2,['Action'],2010,9.99,Valve
2315217,76561198019112245,320,Half-Life 2: Deathmatch,11,True,0,2,['Action'],2004,4.99,Valve
2315218,76561198019112245,340,Half-Life 2: Lost Coast,50,True,0,2,['Action'],2005,4.99,Valve
2315219,76561198019112245,730,Counter-Strike: Global Offensive,15498,True,0,2,['Action'],2012,14.99,Valve
2315220,76561198019112245,236390,War Thunder,83388,True,0,2,"['Free to Play', 'World War II', 'Simulation',...",0,0.00,Dev-desconocidos
2315221,76561198019112245,248570,Toribash,309,True,0,2,"['Action', 'Free to Play', 'Indie', 'Strategy']",2006,0.00,Nabi Studios
2315222,76561198019112245,253710,theHunter,91494,True,0,2,"['Action', 'Adventure', 'Free to Play', 'Simul...",2014,0.00,"Expansive Worlds,Avalanche Studios"
2315223,76561198019112245,107410,Arma 3,2280,True,0,2,"['Action', 'Simulation', 'Strategy']",2013,39.99,Bohemia Interactive
2315224,76561198019112245,301520,Robocraft,0,False,0,1,"['Action', 'Free to Play', 'Indie', 'Massively...",2017,0.00,Freejam


{'Singleplayer', 'Time Manipulation', 'Early Access', 'RTS', 'Steampunk', 'Physics', 'Pirates', 'Abstract', 'Cyberpunk', 'Magic', 'Rhythm', 'Documentary', 'Classic', 'Fighting', '3D Platformer', 'Voxel', 'Video Production', 'Action-Adventure', 'Politics', 'Sports', 'JRPG', 'Turn-Based', 'Colorful', 'Simulation', 'Rogue-lite', 'Remake', 'Local Co-Op', 'Desconocido', 'Dragons', 'Episodic', 'GameMaker', 'Tanks', 'Psychedelic', 'Stylized', 'Tower Defense', 'Mars', 'Experimental', 'Based On A Novel', 'Asynchronous Multiplayer', 'Destruction', 'Education', 'Horror', 'Fantasy', 'Dark Fantasy', 'Romance', 'Dinosaurs', 'Nudity', 'Dating Sim', 'Illuminati', 'Quick-Time Events', 'Management', 'Racing', 'Interactive Fiction', 'Photo Editing', 'Grid-Based Movement', 'Time Attack', 'Sandbox', 'Cartoon', 'Linear', 'Naval', 'Game Development', 'Tactical', 'Historical', 'Top-Down Shooter', 'Split Screen', 'Great Soundtrack', 'Realistic', 'Family Friendly', 'Chess', 'Science', 'Relaxing', 'Mature', 'Online Co-Op', 'Nonlinear', 'Utilities', 'Violent', 'Free to Play', 'Procedural Generation', 'Controller', 'Thriller', 'Side Scroller', 'Sniper', 'Werewolves', 'Turn-Based Tactics', 'Gun Customization', 'Martial Arts', 'Perma Death', 'Hack and Slash', 'World War II', 'Action', 'Puzzle', 'Atmospheric', 'Basketball', 'Crime', 'Robots', 'FMV', 'Flight', 'Parody', 'Top-Down', 'Narration', 'Dark Humor', 'VR', 'Grand Strategy', 'Strategy', 'Sci-fi', 'Philisophical', 'Design & Illustration', 'Sexual Content', 'Comic Book', 'Platformer', 'Runner', 'Action RPG', 'Hidden Object', 'FPS', 'Pool', 'Third-Person Shooter', 'Steam Machine', 'Gore', 'Time Travel', 'Twin Stick Shooter', 'Surreal', 'Animation &amp; Modeling', 'Golf', 'Pinball', 'Animation & Modeling', 'TrackIR', 'Funny', 'Trading', 'Music', 'Psychological Horror', 'Gaming', 'Comedy', 'Retro', 'Software Training', 'Turn-Based Strategy', 'Artificial Intelligence', 'Minimalist', 'Political', 'Hacking', '4 Player Local', 'Satire', 'Card Game', 'Software', 'Dark Comedy', 'Arena Shooter', 'Supernatural', 'PvE', 'Story Rich', 'Anime', 'Fishing', 'Massively Multiplayer', 'Programming', 'RPG', 'Soundtrack', "Shoot 'Em Up", 'Military', 'Co-op', 'Indie', 'Casual', 'Foreign', 'Competitive', 'Cute', 'Horses', 'Games Workshop', 'Hardware', 'Rogue-like', 'Character Customization', 'Audio Production', 'e-sports', 'Third Person', '360 Video', 'Cartoony', 'Mini Golf', 'Cinematic', 'Female Protagonist', "1990's", 'Base Building', 'Web Publishing', 'Fast-Paced', 'Shooter', 'Heist', 'Parkour', 'Post-apocalyptic', 'Tutorial', 'Board Game', 'Mystery', 'PvP', 'Football', '2D', 'MOBA', 'Multiple Endings', 'Sailing', 'Zombies', 'Point & Click', 'Driving', 'Offroad', 'Level Editor', 'Dynamic Narration', 'Team-Based', 'Lovecraftian', 'Dungeon Crawler', 'Visual Novel', 'Hand-drawn', 'Space Sim', 'Moddable', 'Dark', 'Score Attack', 'Star Wars', 'Soccer', 'First-Person', 'Stealth', 'Exploration', 'Survival Horror', 'Survival', 'Short', 'Drama', 'Bullet Hell', 'Choices Matter', 'Real-Time', 'Memes', 'Design &amp; Illustration', 'Replay Value', 'Pixel Graphics', 'Multiplayer', 'Futuristic', 'Puzzle-Platformer', 'Noir', 'Kickstarter', 'God Game', 'Lara Croft', 'War', 'On-Rails Shooter', 'Open World', 'Difficult', 'Wargame', 'Arcade', 'World War I', 'Crafting', '1980s', 'Aliens', '6DOF', 'Local Multiplayer', 'Movie', 'Adventure', 'Walking Simulator', 'Space', '3D Vision', 'Underwater', 'Building', 'Music-Based Procedural Generation', 'Feature Film'}

In [ ]:
# ver genres unicos
df_final['genres'].unique()


array(["['Action']", "['Strategy']", "['Desconocido']", ...,
       "['Casual', 'Simulation', 'Psychological Horror', 'Horror', 'VR']",
       "['Free to Play', 'MOBA', 'Strategy', 'Multiplayer', 'Team-Based', 'Action', 'e-sports', 'Online Co-Op', 'Competitive', 'PvP', 'RTS', 'Difficult', 'RPG', 'Fantasy', 'Tower Defense', 'Co-op', 'Character Customization', 'Replay Value', 'Action RPG', 'Simulation']",
       "['Action', 'Adventure', 'Massively Multiplayer', 'RPG', 'Simulation', 'Strategy']"],
      dtype=object)

In [ ]:
# prompt: release_date unicos ordenados

df_final['release_date'].unique()


array([2000, 1999, 2003, 2001, 1998, 2010, 2004, 2005,    0, 2006, 2007,
       2016, 2008, 1997, 2009, 2002, 2011, 2014, 2012, 2013, 2015, 1994,
       1995, 1996, 1993, 1990, 2017, 1991, 1992, 1988, 1983, 1984, 1987,
       1989, 2018])

In [ ]:
df_final

,user_id,item_id,item_name,playtime_forever,recommend,year,sentiment_analysis,genres,release_date,price,developer
0,76561197970982479,10,Counter-Strike,6,True,0,2,['Action'],2000,9.99,Valve
1,76561197970982479,20,Team Fortress Classic,0,False,0,1,['Action'],1999,4.99,Valve
2,76561197970982479,30,Day of Defeat,7,True,0,2,['Action'],2003,4.99,Valve
3,76561197970982479,40,Deathmatch Classic,0,False,0,1,['Action'],2001,4.99,Valve
4,76561197970982479,50,Half-Life: Opposing Force,0,False,0,1,['Action'],1999,4.99,Gearbox Software
...,...,...,...,...,...,...,...,...,...,...,...
5100269,Fuckfhaisjnsnsjakaka,730,Counter-Strike: Global Offensive,0,True,0,2,['Action'],2012,14.99,Valve
5100270,KinkyyyCSGO,282660,EasyAntiCheat eSports,0,True,0,2,['Utilities'],2006,0.00,EasyAntiCheat Ltd
5100271,LydiaMorley,273110,Counter-Strike Nexon: Zombies,0,True,0,2,"['Action', 'Casual', 'Free to Play', 'Massivel...",2014,0.00,Nexon
5100272,LydiaMorley,730,Counter-Strike: Global Offensive,0,True,0,2,['Action'],2012,14.99,Valve


Funcion **1**

In [ ]:
def developer(desarrollador):
    """
    Esta función toma como entrada el nombre de un desarrollador y devuelve un diccionario que contiene información sobre los juegos lanzados por ese desarrollador.
    La información incluye el número total de juegos lanzados por año, el número de juegos gratuitos lanzados por año y el porcentaje de juegos gratuitos lanzados por año.

    Parámetros:
    desarrollador (str): El nombre del desarrollador.

    Devuelve:
    dict_developer (dict): Un diccionario que contiene información sobre los juegos lanzados por el desarrollador.

    Ejemplo:
    >>> developer("Nombre del Desarrollador")
    [{'Año': 2010, 'Cantidad de artículos': 5, 'Contenidos Gratis': 2, 'Porcentaje Gratis': '40%'},
     {'Año': 2011, 'Cantidad de artículos': 10, 'Contenidos Gratis': 5, 'Porcentaje Gratis': '50%'},
     {'Año': 'Desconocido', 'Cantidad de artículos': 1, 'Contenidos Gratis': 0, 'Porcentaje Gratis': '0%'}]

    En caso de que el desarrollador no exista, devuelve un diccionario con un mensaje de error.

    Ejemplo:
    >>> developer("Desarrollador Inexistente")
    {
        "detail": [
            {
                "loc": ["string", 0],
                "msg": "El desarrollador no existe",
                "type": "value_error"
            }
        ]
    }
    """
    # Filtrar el dataframe por el desarrollador dado
    df_dev = df_final[df_final['developer'] == desarrollador]

    # Si el dataframe está vacío, lanzar una excepción
    if df_dev.empty:
        return {
            "detail": [
                {
                    "loc": ["string", 0],
                    "msg": "El desarrollador no existe",
                    "type": "value_error"
                }
            ]
        }

    # Agrupar por año y contar el número total de juegos y el número de juegos gratuitos
    juegos_por_año = df_dev.groupby('release_date').size()
    juegos_gratis_por_año = df_dev[df_dev['price'] == 0.0].groupby('release_date').size()

    # Asegurarse de que juegos_gratis_por_año tenga la misma longitud que juegos_por_año
    juegos_gratis_por_año = juegos_gratis_por_año.reindex(juegos_por_año.index, fill_value=0)

    # Calcular el porcentaje de juegos gratis y redondearlo al entero más cercano
    porcentaje_gratis = ((juegos_gratis_por_año / juegos_por_año) * 100).round().astype(int)

    # Asignar nombres a las series
    juegos_por_año.name = "Cantidad de artículos"
    juegos_gratis_por_año.name = "Contenidos Gratis"
    porcentaje_gratis.name = "Porcentaje Gratis"

    # Unir las series en un DataFrame
    tabla = pd.concat([juegos_por_año, juegos_gratis_por_año, porcentaje_gratis], axis=1).reset_index()
    tabla.columns = ['Año', 'Cantidad de artículos', 'Contenidos Gratis', 'Porcentaje Gratis']

    # Reemplazar 0 con 'Desconocido' en la columna 'Año'
    tabla['Año'] = tabla['Año'].replace(0, 'Desconocido')

    # Agregar el signo de porcentaje a 'Porcentaje Gratis'
    tabla['Porcentaje Gratis'] = tabla['Porcentaje Gratis'].apply(lambda x: f"{x}%")

    # Convertir el DataFrame a un diccionario
    dict_developer = tabla.to_dict(orient='records')

    return dict_developer

In [ ]:
# llamar a la función
developer("Valve")

[{'Año': 1998,
  'Cantidad de artículos': 7669,
  'Contenidos Gratis': 0,
  'Porcentaje Gratis': '0%'},
 {'Año': 1999,
  'Cantidad de artículos': 6270,
  'Contenidos Gratis': 0,
  'Porcentaje Gratis': '0%'},
 {'Año': 2000,
  'Cantidad de artículos': 12898,
  'Contenidos Gratis': 0,
  'Porcentaje Gratis': '0%'},
 {'Año': 2001,
  'Cantidad de artículos': 3242,
  'Contenidos Gratis': 0,
  'Porcentaje Gratis': '0%'},
 {'Año': 2003,
  'Cantidad de artículos': 3431,
  'Contenidos Gratis': 0,
  'Porcentaje Gratis': '0%'},
 {'Año': 2004,
  'Cantidad de artículos': 71279,
  'Contenidos Gratis': 0,
  'Porcentaje Gratis': '0%'},
 {'Año': 2005,
  'Cantidad de artículos': 19612,
  'Contenidos Gratis': 0,
  'Porcentaje Gratis': '0%'},
 {'Año': 2006,
  'Cantidad de artículos': 26316,
  'Contenidos Gratis': 0,
  'Porcentaje Gratis': '0%'},
 {'Año': 2007,
  'Cantidad de artículos': 31823,
  'Contenidos Gratis': 323,
  'Porcentaje Gratis': '1%'},
 {'Año': 2008,
  'Cantidad de artículos': 6530,
  'Conten

Funcion **2**

In [ ]:
def user_data(user_id):
    """
    Esta función devuelve la cantidad de dinero gastado por el usuario, el porcentaje de recomendación
    basado en las revisiones (recommend) y la cantidad de artículos.

    Args:
    - User_id (str): El ID del usuario para el cual se desea obtener la información.

    Returns:
    - dict: Un diccionario que contiene la información del usuario, incluyendo:
        - "Usuario": El ID del usuario.
        - "Dinero gastado": La cantidad de dinero gastado por el usuario en USD.
        - "% de recomendación": El porcentaje de recomendación basado en las revisiones.
        - "cantidad de artículos": La cantidad de artículos únicos que el usuario ha revisado.

    Ejemplo:
    >>> user_data("ID_Usuario")
    {
        'Usuario': 'ID_Usuario',
        'Dinero gastado': '35.0 USD',
        '% de recomendación': '60%',
        'cantidad de artículos': 5
    }

    En caso de que el usuario no exista, devuelve un diccionario con un mensaje de error.

    Ejemplo:
    >>> user_data("Usuario_Inexistente")
    {
        "detail": [
            {
                "loc": ["string", 0],
                "msg": "El usuario Usuario_Inexistente no existe.",
                "type": "value_error"
            }
        ]
    }
    """
    # Filtrar el dataframe 'df_final' por el 'user_id' dado
    df_filtered = df_final[df_final['user_id'] == user_id]

    # Verificar si el dataframe filtrado está vacío
    if df_filtered.empty:
        return {
            "detail": [
                {
                    "loc": ["string", 0],
                    "msg": f"El usuario {user_id} no existe.",
                    "type": "value_error"
                }
            ]
        }

    # Calcular la cantidad de dinero gastado y redondearlo a dos decimales
    total_spent = round(df_filtered['price'].sum(), 2)

    # Calcular el porcentaje de recomendación
    total_reviews = df_filtered['recommend'].count()
    recommended_reviews = df_filtered[df_filtered['recommend'] == True]['recommend'].count()
    recommendation_percentage = round((recommended_reviews / total_reviews) * 100) if total_reviews > 0 else 0

    # Calcular la cantidad de items
    total_items = df_filtered['item_name'].nunique()

    # Crear el diccionario de resultados
    dict_userdata = {
        "Usuario": user_id,
        "Dinero gastado": f"{total_spent} USD",
        "% de recomendación": f"{recommendation_percentage}%",
        "cantidad de artículos": total_items
    }

    return dict_userdata

In [ ]:
# llamar a la función
user_data("76561197970982479")

{'Usuario': '76561197970982479',
 'Dinero gastado': '3434.29 USD',
 '% de recomendación': '71%',
 'cantidad de artículos': 276}

In [ ]:
# buscar usuario 76561197970982479 en el dataframe
df_final[df_final['user_id'] == '76561197970982479']

,user_id,item_id,item_name,playtime_forever,recommend,year,sentiment_analysis,genres,release_date,price,developer
0,76561197970982479,10,Counter-Strike,6,True,0,2,['Action'],2000,9.99,Valve
1,76561197970982479,20,Team Fortress Classic,0,False,0,1,['Action'],1999,4.99,Valve
2,76561197970982479,30,Day of Defeat,7,True,0,2,['Action'],2003,4.99,Valve
3,76561197970982479,40,Deathmatch Classic,0,False,0,1,['Action'],2001,4.99,Valve
4,76561197970982479,50,Half-Life: Opposing Force,0,False,0,1,['Action'],1999,4.99,Gearbox Software
...,...,...,...,...,...,...,...,...,...,...,...
272,76561197970982479,289130,Endless Legend,593,True,0,2,"['RPG', 'Strategy']",2014,29.99,AMPLITUDE Studios
273,76561197970982479,268500,XCOM 2,1495,True,0,2,['Strategy'],2016,59.99,"Firaxis,Feral Interactive (Mac),Feral Interact..."
274,76561197970982479,461640,Sins Of The Demon RPG,0,False,0,1,"['Action', 'Adventure', 'Indie', 'RPG']",2016,1.99,Chandler Rounsley
275,76561197970982479,379720,DOOM,793,True,0,2,['Action'],2016,29.99,id Software


In [ ]:
# prompt: contar la cantidad de item_id unicos del user_id 76561197970982479

unique_items = df_final[df_final['user_id'] == '76561197970982479']['item_id'].nunique()
print(f"El usuario 76561197970982479 tiene {unique_items} item_id únicos.")


El usuario 76561197970982479 tiene 277 item_id únicos.


Sistemas de recomendación

In [ ]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163017 sha256=b51529e7808e4d425b2d91829c504b7c0df17b064d17a197354561284d11c489
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [1]:
import pandas as pd
import pyarrow.parquet as pq
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split, GridSearchCV
from surprise import Dataset, Reader, SVD
import numpy as np

In [ ]:
import gzip

In [ ]:
# prompt: cargar los datos desde el archivo joblib
import joblib
from joblib import load

model = joblib.load('/content/item_similarity_df.joblib')


In [ ]:
# Guardar el modelo comprimido en un nuevo archivo utilizando xz con un nivel de compresión más alto
joblib.dump(model, 'items_similar.joblib', compress=('xz', 9))

['items_similar.joblib']

In [17]:
df_final = pd.read_parquet('/content/drive/MyDrive/Dict/df_final.parquet')

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
from joblib import dump
import ast

# Convertir la columna de géneros a una lista de géneros
# df_modelo['genres'] = df_modelo['genres'].apply(ast.literal_eval)

# Agrupar por item_id y combinar los géneros en una lista única
df_grouped = df_modelo.groupby('item_id')['genres'].sum().reset_index()

# Crear el codificador One-Hot
mlb = MultiLabelBinarizer()

# Aplicar el codificador One-Hot a la columna de géneros
one_hot_genres = mlb.fit_transform(df_grouped['genres'])

# Crear un DataFrame con las columnas codificadas
df_genres = pd.DataFrame(one_hot_genres, columns=mlb.classes_, index=df_grouped['item_id'])

# Calcular la similitud del coseno entre los ítems
item_similarity = cosine_similarity(df_genres)

# Convertir el resultado en un DataFrame para facilitar su manejo
item_similarity_df = pd.DataFrame(item_similarity, index=df_genres.index, columns=df_genres.index)

# Guardar el DataFrame en un archivo parquet
dump(item_similarity_df, "item_similarity_df.joblib")

['item_similarity_df.joblib']

Opcion 2

In [3]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Convertir la columna de géneros a una lista de géneros
# df_modelo['genres'] = df_modelo['genres'].apply(ast.literal_eval)

# Agrupar por item_id y combinar los géneros en una lista única
df_grouped = df_modelo.groupby('item_id')['genres'].sum().reset_index()

# Crear el codificador One-Hot
mlb = MultiLabelBinarizer()

# Aplicar el codificador One-Hot a la columna de géneros
one_hot_genres = mlb.fit_transform(df_grouped['genres'])

# Crear un DataFrame con las columnas codificadas
df_genres = pd.DataFrame(one_hot_genres, columns=mlb.classes_, index=df_grouped['item_id'])

# Calcular la similitud del coseno entre los ítems
item_similarity = cosine_similarity(df_genres)

# Convertir el resultado en un DataFrame para facilitar su manejo
item_similarity_df = pd.DataFrame(item_similarity, index=df_genres.index, columns=df_genres.index)

# Convertir los nombres de las columnas a tipo string
item_similarity_df.columns = item_similarity_df.columns.astype(str)

# Guardar el DataFrame en un archivo Parquet
item_similarity_df.to_parquet("item_similarity_df.parquet")

In [12]:
from joblib import load
# SISTEMA DE RECOMENDACIÓN - ITEM-ITEM
def recomendacion_juego(item_id):
    """
    Sistema de recomendación item-item.

    Ingresando el ID de un producto, se devuelve una lista con 5 juegos recomendados similares al ingresado.

    Args:
    - item_id: El ID del producto para el cual se desean obtener recomendaciones.

    Returns:
    - dict: Un diccionario que contiene una lista de 5 juegos recomendados similares al ingresado.
      El diccionario tiene el formato {"Juegos similares al ID ingresado": {ID1: 'Nombre1', ID2: 'Nombre2', ...}}.

    Raises:
    - Si el DataFrame está vacío, devuelve un mensaje de error.
    - Si el item_id no existe en el DataFrame, devuelve un mensaje de error.
    - Si ocurre un error al cargar el archivo de similitud, intenta calcular la similitud del coseno
      entre los productos y proporciona recomendaciones basadas en eso.
    """
    # Leer el archivo parquet en un DataFrame
    data = df_modelo

    try:
        # Cargar la matriz de similitud de items_similarity
        item_similarity_df = load('/content/items_similar.joblib')

        # mensaje de test para verificar que se cargó la matriz precalculada
        print("Se cargó la matriz precalculada.")

    except Exception as e:
        print(f"Ocurrió un error al cargar el archivo: {e}")

    # Si el dataframe está vacío, devolver un mensaje de error
    if data.empty:
        return {
            "detail": [
                {
                    "loc": ["string", 0],
                    "msg": "El DataFrame está vacío",
                    "type": "value_error"
                }
            ]
        }

    # Si el item_id no existe en el DataFrame, devolver un mensaje de error
    if item_id not in data['item_id'].values:
        return {
            "detail": [
                {
                    "loc": ["string", 0],
                    "msg": "El item_id proporcionado no existe",
                    "type": "value_error"
                }
            ]
        }

    # Obtener los juegos similares al juego dado
    similar_games = item_similarity_df[item_id].sort_values(ascending=False)

    # Obtener los 5 juegos más similares
    similar_games_ids = similar_games.head(6).index.tolist()[1:]  # Excluimos el primer juego porque es el mismo juego

    # Crear un diccionario con los ids y los nombres de los juegos
    recommended_games = {game_id: data[data['item_id'] == game_id]['item_name'].iloc[0] for game_id in similar_games_ids}

    return {"Juegos similares al id ingresado": recommended_games}

In [13]:
# llamado a la función
recomendacion_juego(333930)

Se cargó la matriz precalculada.


{'Juegos similares al id ingresado': {17080: 'Tribes: Ascend',
  409510: 'Genesis Online',
  244630: 'NEOTOKYOÂ°',
  206210: 'Gotham City Impostors: Free To Play',
  440: 'Team Fortress 2'}}

In [10]:
item_similarity_df = pd.read_parquet('/content/item_similarity_df.parquet')

In [ ]:
# llamado a la función
recomendacion_juego(333930)

Se cargó la matriz precalculada.


{'Juegos similares al id ingresado': {17080: 'Tribes: Ascend',
  409510: 'Genesis Online',
  244630: 'NEOTOKYOÂ°',
  206210: 'Gotham City Impostors: Free To Play',
  440: 'Team Fortress 2'}}

Modelo user-item

In [ ]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split, GridSearchCV
from joblib import dump

# Tomar una muestra del 2% de los datos
df_modelo = df_modelo.sample(frac=0.02, random_state=42)

# Reemplazar los valores de la columna 'recommend'
df_modelo['recommend'] = df_modelo['recommend'].replace({True: 1, False: 0})

# Crear la columna 'rating'
df_modelo['rating'] = 0
df_modelo.loc[(df_modelo['sentiment_analysis'] == 2) & (df_modelo['recommend'] == 1), 'rating'] = 5
df_modelo.loc[(df_modelo['sentiment_analysis'] == 2) & (df_modelo['recommend'] == 0), 'rating'] = 4
df_modelo.loc[(df_modelo['sentiment_analysis'] == 1) & (df_modelo['recommend'] == 1), 'rating'] = 3
df_modelo.loc[(df_modelo['sentiment_analysis'] == 1) & (df_modelo['recommend'] == 0), 'rating'] = 2
df_modelo.loc[(df_modelo['sentiment_analysis'] == 0) & (df_modelo['recommend'] == 1), 'rating'] = 1

# Guardar el DataFrame en un nuevo archivo .parquet
df_modelo.to_parquet('user_item_modelo.parquet')

# Cargar los datos del archivo .parquet
new_df = pd.read_parquet('user_item_modelo.parquet')

# Adaptar rating_scale a la escala de datos
reader = Reader(rating_scale=(0, 5))

# Cargar los datos en un objeto Dataset de Surprise
data = Dataset.load_from_df(new_df[['user_id', 'item_name', 'rating']], reader)

# Dividir el conjunto de datos en entrenamiento y prueba
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Optimizar los hiperparámetros con GridSearchCV
param_grid = {'n_factors': [5, 50, 100], 'n_epochs': [5, 10, 20], 'lr_all': [0.001, 0.002, 0.005], 'reg_all': [0.002, 0.02, 0.2]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=5, n_jobs=-1)
gs.fit(data)

# Entrenar el modelo SVD con los hiperparámetros optimizados
model = SVD(**gs.best_params['rmse'])
model.fit(trainset)

# Guardar el modelo en un archivo joblib
dump(model, 'user_item_model.joblib')

In [18]:
# prompt: developer unicos de df_final

df_final['developer'].unique()


array(['Valve', 'Gearbox Software', 'Double Fine Productions', ...,
       'Adam Foster', 'Team Chivalry', 'Empires'], dtype=object)

In [19]:
# prompt: user_id unicos

user_id_unicos = df_final['user_id'].unique()
print(user_id_unicos)


['76561197970982479' 'js41637' 'evcentric' ... 'Fuckfhaisjnsnsjakaka'
 'KinkyyyCSGO' 'LydiaMorley']


In [20]:
# prompt: genres unicos

df_final['genres'].unique()


array(["['Action']", "['Strategy']", "['Desconocido']", ...,
       "['Casual', 'Simulation', 'Psychological Horror', 'Horror', 'VR']",
       "['Free to Play', 'MOBA', 'Strategy', 'Multiplayer', 'Team-Based', 'Action', 'e-sports', 'Online Co-Op', 'Competitive', 'PvP', 'RTS', 'Difficult', 'RPG', 'Fantasy', 'Tower Defense', 'Co-op', 'Character Customization', 'Replay Value', 'Action RPG', 'Simulation']",
       "['Action', 'Adventure', 'Massively Multiplayer', 'RPG', 'Simulation', 'Strategy']"],
      dtype=object)

In [21]:
# prompt: release_date unicos

df_final['release_date'].unique()


array([2000, 1999, 2003, 2001, 1998, 2010, 2004, 2005,    0, 2006, 2007,
       2016, 2008, 1997, 2009, 2002, 2011, 2014, 2012, 2013, 2015, 1994,
       1995, 1996, 1993, 1990, 2017, 1991, 1992, 1988, 1983, 1984, 1987,
       1989, 2018])

In [22]:
# prompt: item_id unicos

df_final['item_id'].unique()


array([    10,     20,     30, ..., 225833,  22110, 414390])